In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

import os
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns',5000)

In [145]:
#printing filenames
for filename in os.listdir('./data/'):
    if filename.endswith('.csv'):
        print(filename)

print('-'*20)

for filename in os.listdir('./data/train_AUpWtIz/'):
    if filename.endswith('.csv'):
        print(filename)

sample_submission_Byiv0dS.csv
test_QyjYwdj.csv
--------------------
campaign_data.csv
coupon_item_mapping.csv
customer_demographics.csv
customer_transaction_data.csv
item_data.csv
train.csv


In [147]:
#reading files
train = pd.read_csv('./data/train_AUpWtIz/train.csv')
campaign = pd.read_csv('./data/train_AUpWtIz/campaign_data.csv')
items = pd.read_csv('./data/train_AUpWtIz/item_data.csv')
coupons = pd.read_csv('./data/train_AUpWtIz/coupon_item_mapping.csv')
cust_demo = pd.read_csv('./data/train_AUpWtIz/customer_demographics.csv')
cust_tran = pd.read_csv('./data/train_AUpWtIz/customer_transaction_data.csv')

test = pd.read_csv('./data/test_QyjYwdj.csv')
sample = pd.read_csv('./data/sample_submission_Byiv0dS.csv')

In [148]:
########################################## Campaign ##########################################

In [149]:
#todatetime
campaign['start_date'] = pd.to_datetime(campaign['start_date'], format = '%d/%m/%y')
campaign['end_date'] = pd.to_datetime(campaign['end_date'], format = '%d/%m/%y')

In [150]:
#adding campaign type to train and test
train['campaign_type'] = train.campaign_id.map(campaign.groupby('campaign_id').campaign_type.apply(lambda x: x.unique()[0]))
test['campaign_type'] = test.campaign_id.map(campaign.groupby('campaign_id').campaign_type.apply(lambda x: x.unique()[0]))

In [151]:
########################################## Customer demographics ##########################################

In [152]:
#type of family size, no of children = int64
cust_demo['family_size'] = cust_demo.family_size.apply(lambda x: int(re.sub('\+','',x)))
cust_demo['no_of_children'] = cust_demo.no_of_children.apply(lambda x: float(re.sub('\+','',x)) if pd.notna(x) else x)

In [153]:
#Filling nans marital_status

#customers with family size =1 will be single
cust_demo.loc[pd.isnull(cust_demo.marital_status) & (cust_demo.family_size == 1),'marital_status'] = 'Single'

#customers whos fam size - no of childrens == 1, will also be single
cust_demo.loc[(cust_demo.family_size - cust_demo.no_of_children == 1) & pd.isnull(cust_demo.marital_status),'marital_status'] = 'Single'

#from the orignal data we have 142 of 152 customers with diff of 2 in their fam size and #childrens are Married
cust_demo.loc[(pd.isnull(cust_demo.marital_status)) & ((cust_demo.family_size - cust_demo.no_of_children) == 2)  & (pd.notnull(cust_demo.no_of_children)),'marital_status'] = 'Married'

#original data shows customers with fam size == 2, and nans in no of childrens are majorly Married
cust_demo.loc[pd.isnull(cust_demo.marital_status) & (pd.isnull(cust_demo.no_of_children)) & (cust_demo.family_size ==2),'marital_status'] = 'Married'

In [154]:
#Filling nans in no of children

#Married people with family_size ==2 will have 0 childrens
cust_demo.loc[pd.isnull(cust_demo.no_of_children) & (cust_demo.marital_status == 'Married') & (cust_demo.family_size == 2),'no_of_children'] = 0.0

#customers with family size 1 will have zero childrens
cust_demo.loc[pd.isnull(cust_demo.no_of_children) & (cust_demo.family_size == 1), 'no_of_children'] = 0.0

#singles with family size == 2, will probably have 1 child
cust_demo.loc[pd.isnull(cust_demo.no_of_children) & (cust_demo.family_size == 2),'no_of_children'] = 1.0

In [155]:
########################################## Customer transactions ##########################################

In [156]:
cust_tran['date'] = pd.to_datetime(cust_tran['date'])

In [157]:
########################################## common ########################################## 

In [158]:
#merging train test and cust_demo on campaign_id
train = pd.merge(train,cust_demo, on='customer_id', how='left')
test = pd.merge(test,cust_demo, on='customer_id', how='left')

In [159]:
############# Intersection between Items bought by customer and items available in coupon

In [160]:
#cust2items
cust_tran['str_item'] = cust_tran.item_id.apply(lambda x: str(x)) #did this to calculate d_cust2items, no need further
d_cust2items = cust_tran.groupby('customer_id').str_item.apply(lambda x: ' '.join(x)).to_dict()
cust_tran.drop('str_item',axis=1,inplace=True)

In [161]:
#coupon2items
d_coupon2items = coupons.groupby('coupon_id').item_id.apply(lambda x: ' '.join(list(x.apply(lambda x: str(x))))).to_dict()

In [162]:
#intersect of cust2items and coupon2items (increased score by 0.14)
train['bought_X_vailable'] = train[['coupon_id','customer_id']].apply(lambda x : len(np.intersect1d(d_cust2items[x[1]].split() , d_coupon2items[x[0]].split())) , axis=1)
test['bought_X_vailable'] = test[['coupon_id','customer_id']].apply(lambda x : len(np.intersect1d(d_cust2items[x[1]].split() , d_coupon2items[x[0]].split())) , axis=1)

In [70]:
#############

In [71]:
#item2coupons
d_item2coupons = coupons.groupby('item_id').coupon_id.apply(lambda x: ' '.join(list(x.apply(lambda x: str(x))))).to_dict()

In [72]:
#adding col for whether coupon was applied on that item (i.e redeemed or not)
cust_tran['redeem'] = cust_tran.coupon_discount.apply(lambda x: 1 if x<0 else 0)

In [73]:
##############  1.) Calculating redeemed % per item from cust_tran
#               2.) Summing all those per for items in a coupon, take mean finally
#               3.) map it to coupons

In [74]:
#getting dict for redeem perc per item from cust tran
d_per_item_redeemed_history = ((cust_tran.groupby('item_id').redeem.sum() / cust_tran.groupby('item_id').redeem.count()) *100).to_dict()

In [75]:
#some items corresponding to test coupons are not in d_per_item_redeemed_hist hence need for this func
def item_redeem_func(x):
    for item in d_coupon2items[x].split():
        per = []
        try:
            per.append(d_per_item_redeemed_history[int(item)])

        except:
            pass
    k = [np.mean(per) if pd.isna(np.mean(per)) == False else 0]
    return k[0]

In [76]:
#applying the above func to coupon_id
train['item_redeem'] = train.coupon_id.apply(item_redeem_func)
test['item_redeem'] = test.coupon_id.apply(item_redeem_func)

In [77]:
###############

In [78]:
############### redeem per acco to cust (same thing as above, but here wrt customer_id)

In [79]:
#how have customers taken the discount
d_per_cust_redeem_history = ((cust_tran.groupby('customer_id').redeem.sum() / cust_tran.groupby('customer_id').redeem.count())*100).to_dict()

In [80]:
#adding a col for cust redeem #increased score by 0.03
train['cust_redeem'] = train.customer_id.map(d_per_cust_redeem_history)
test['cust_redeem'] = test.customer_id.map(d_per_cust_redeem_history)

In [81]:
###############

In [82]:
##############

In [83]:
#adding net price
cust_tran['net_price'] = cust_tran['selling_price'] - cust_tran['other_discount'] - cust_tran['coupon_discount']

#dict for cust_id to income bracket
d_cust2_incomebrac = cust_demo[['customer_id','income_bracket']].set_index('customer_id').to_dict()['income_bracket']

#adding income bracket col in customer trans
cust_tran['income_bracket'] = cust_tran.customer_id.map(d_cust2_incomebrac)

#merging cust_trans with items on item_id
cust_tran = pd.merge(cust_tran, items, how='left', on='item_id')

In [84]:
#################### redeem history mapped to cust based on category

In [85]:
#redeem history based on category
d_per_cat_redeem_history = (cust_tran.groupby('category').redeem.sum() / cust_tran.groupby('category').redeem.count()*1000).to_dict()

In [86]:
#(increased score by 0.0001)
d_per_cust_redeem_history_catwali = cust_tran.groupby('customer_id').category.apply(lambda x: np.mean([d_per_cat_redeem_history[k] for k in x.values]))

train['cat_cust_redeem'] = train.customer_id.map(d_per_cust_redeem_history_catwali)
test['cat_cust_redeem'] = test.customer_id.map(d_per_cust_redeem_history_catwali)

In [87]:
####################

In [88]:
############ if for a customer brands bought by him are available in the coupon given, high chance of redeem

In [89]:
#dict for all brands per customer
d_cust2brands = cust_tran.groupby('customer_id').brand.apply(lambda x: ' '.join([str(k) for k in x.unique()])).to_dict()

In [90]:
#dict for item 2 brand
d_item2brand = cust_tran.groupby('item_id').brand.apply(lambda x: x.unique()[0]).to_dict()

In [91]:
coupons['brand'] = coupons.item_id.map(d_item2brand).fillna('99999999999')

In [92]:
#dict for item 2 brand
d_coupon2brands = coupons.groupby('coupon_id').brand.apply(lambda x: ' '.join([str(int(k)) for k in x.unique()])).to_dict()

In [93]:
#adding col of intersection of cust2brand and item2brand
train['brand_bot'] = train[['customer_id','coupon_id']].apply(lambda x: len(np.intersect1d(d_cust2brands[x[0]].split(), d_coupon2brands[x[1]].split())), axis=1)
test['brand_bot'] = test[['customer_id','coupon_id']].apply(lambda x: len(np.intersect1d(d_cust2brands[x[0]].split(), d_coupon2brands[x[1]].split())), axis=1)

In [95]:
###################

In [96]:
################### Imputing rented and Age

In [97]:
#filling nans in train.rented with 2
train.rented.fillna(2,inplace=True)
test.rented.fillna(2,inplace=True)

In [98]:
#imputing age_range based on campaign_id

def d_age(df):
    k = df.groupby('campaign_id').age_range.value_counts()
    k = k.reset_index(name='value').sort_values(['campaign_id','value'], ascending=[True,False])
    d_age = {}
    for i in list(df.campaign_id.unique()):
        df = k.loc[k.campaign_id == i,['age_range','value']]
        df = df.set_index('age_range')
        max_val_per_campaign = df.idxmax().value
        d_age[i] = max_val_per_campaign
        
    return d_age

    
#filling nans with d_age
train.loc[(pd.isnull(train.age_range)),'age_range'] = train.loc[(pd.isnull(train.age_range)),'campaign_id'].map(d_age(train))
test.loc[(pd.isnull(test.age_range)),'age_range'] = test.loc[(pd.isnull(test.age_range)),'campaign_id'].map(d_age(test))


In [100]:
###################

In [49]:
#dropping some cols
#train.drop(['marital_status','family_size','no_of_children','income_bracket'],axis=1,inplace=True)
#test.drop(['marital_status','family_size','no_of_children','income_bracket'],axis=1,inplace=True)

In [101]:
#adding brand (most frequent)
train['brand'] = train.coupon_id.map(coupons.groupby('coupon_id').brand.apply(lambda x: x.values[0]).to_dict())
test['brand'] = test.coupon_id.map(coupons.groupby('coupon_id').brand.apply(lambda x: x.values[0]).to_dict())

In [102]:
############################ val set

In [103]:
#col for whether cust and coup is in test or not ---> (in order to make val set)
commom_cust = np.intersect1d(train.customer_id.unique(),test.customer_id.unique())
commom_coup = np.intersect1d(train.coupon_id.unique(),test.coupon_id.unique())

In [104]:
#adding col to see whether cust is in test or not 
train['test_cust'] = train.customer_id.apply(lambda x: 1 if x in commom_cust else 0)
train['test_coup'] = train.coupon_id.apply(lambda x: 1 if x in commom_coup else 0)

In [139]:
####Validation set
# val_index

#(len(train[pd.isnull(train.family_size) & (train.redemption_status == 1)]) / len(train)) * 7837 #16
index1 = train[pd.isnull(train.family_size) & (train.redemption_status == 1) & (train.test_cust == 1) & (train.test_coup == 1)].sample(16, random_state=1996).index

#(len(train[pd.notnull(train.family_size) & (train.redemption_status == 1)]) / len(train) ) * 7837 #57
index2 = train[pd.notnull(train.family_size) & (train.redemption_status == 1) & (train.test_cust == 1) & (train.test_coup == 1)].sample(57, random_state=1996).index

#(len(train[pd.isnull(train.family_size) & (train.redemption_status == 0)]) / len(train)) * 7837 #3455
index3 = train[pd.isnull(train.family_size) & (train.redemption_status == 0) & (train.test_cust == 1) & (train.test_coup == 1)].sample(3366, random_state=1996).index

#(len(train[pd.notnull(train.family_size) & (train.redemption_status == 0)]) / len(train)) * 7837 #4309
index4 = train[pd.notnull(train.family_size) & (train.redemption_status == 0) & (train.test_cust == 1) & (train.test_coup == 1)].sample(4309, random_state=1996).index



#new train and val set
val_index = []
for i in [index1,index2, index3, index4]:
    val_index.extend(i)#main val_index
    
train_index = set(train.index)
train_index = train_index.symmetric_difference(val_index)#main train index

new_train = train.loc[train_index]
val = train.loc[val_index].sample(frac=1, random_state = 1996)
new_test = test

In [140]:
#final_train = new_train.dropna(axis=1).drop(['test_cust','test_coup'], axis=1)
#final_test = new_test.dropna(axis=1)#.drop(['coup_redeem'], axis=1)
#val = val.dropna(axis=1).drop(['test_cust','test_coup'], axis=1)

final_train = train.dropna(axis=1).drop(['test_cust','test_coup'], axis=1)
final_test = test.dropna(axis=1)

################# Label Encoding

#label encoding features
final_train['campaign_type'] = final_train.campaign_type.map({'X':0,'Y':1})
#val['campaign_type'] = val.campaign_type.map({'X':0,'Y':1})
final_test['campaign_type'] = final_test.campaign_type.map({'X':0,'Y':1})

final_train['age_range'] = final_train.age_range.map({'46-55':0,'36-45':1,'18-25':2,'26-35':3,'56-70':4,'70+':5})
#val['age_range'] = val.age_range.map({'46-55':0,'36-45':1,'18-25':2,'26-35':3,'56-70':4,'70+':5})
final_test['age_range'] = final_test.age_range.map({'46-55':0,'36-45':1,'18-25':2,'26-35':3,'56-70':4,'70+':5})

###############

############## train_test

#preparing data
X_train = final_train.drop(['redemption_status'],axis=1)
y_train = final_train.redemption_status

#val_x = val.drop(['redemption_status'],axis=1)
#val_y = val.redemption_status

X_test = final_test

In [141]:
from sklearn.metrics import roc_auc_score

In [143]:
from xgboost import XGBClassifier

preds1= pd.DataFrame()
auc_roc1 = []
val_auc = []

for k in range(0,10):
    df_1 = final_train[final_train.redemption_status == 1]
    df_0 = final_train[final_train.redemption_status == 0].sample(1000, random_state =2*k*k*k)
    
    df = pd.concat([df_0,df_1],axis=0).sample(frac=1)

    X_train = df.drop('redemption_status',axis=1)
    y_train = df.redemption_status

    model1 = XGBClassifier(n_estimators=100, scale_pos_weight=2)
    model1.fit(X_train,y_train)

    #pred by model1
    auc_roc1.append(roc_auc_score(y_train, model1.predict_proba(X_train)[:,1].round(3)))
    preds1['k'+str(k)] = model1.predict_proba(X_test)[:,1].round(3)
    


    print(k, end='')

0123456789

In [138]:
pred = preds1.mean(axis=1)
roc_auc_score(val_y, pred)

0.9729695238945161

In [ ]:
##############

In [144]:
pred_test = preds1.mean(axis=1)

sample['redemption_status'] = pred_test

name = 'sol/bestCols_brandbot_2kkk_brand.csv'
sample.to_csv(name,index=False)